<a href="https://colab.research.google.com/github/gchinca/app_transporte/blob/main/app_transporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **ANÁLISIS DE RENTABILIDAD PLATAFORMA DE TRANSPORTE**

**Escenario 1:**

Va a ser un sábado muy concurrido y será necesario que haya muchos más coches en la carretera que la semana pasada. Para lograr que los conductores se conecten a Internet, estamos evaluando las siguientes dos opciones de bonificación en términos de costo:

**Opción 1:** $50 por cada conductor que esté en línea al menos 8 horas, acepte el 90% de las solicitudes, complete 10 viajes y tenga una calificación de 4.7 o mejor durante el período de tiempo;

**Opción 2:** $4/viaje para todos los conductores que completen 12 viajes y tengan una calificación de 4.7 o mejor.

**Escenario 2:**

**1.** Un taxista actualmente genera $200 por día en tarifas (antes de gastos), trabaja seis días a la semana, se toma tres semanas libres y tiene los siguientes gastos:

Gasolina: $200 por semana

Seguro: $400 por mes

Alquiler de vehículo (por semana): $500

El conductor no paga los gastos de gasolina ni de alquiler en las semanas libres.

**2.** Supongamos que el mismo conductor compraría un Town Car y se asociaría con la plataforma. Si lo hace, sus gastos de gasolina aumentarían un 5%, sus gastos de seguro disminuirían un 20% y ya no alquilaría un vehículo. Sin embargo, necesitaría comprar un coche de $40.000. El conductor seguiría tomándose tres semanas de descanso al año.


## **PREGUNTAS DEL NEGOCIO**

**Escenario 1**

- ¿A cuánto ascendería el pago total del bono con la Opción 1?

- ¿A cuánto ascendería el pago total del bono con la Opción 2?

- ¿Cuántos conductores calificarían para una bonificación según la Opción 1 pero no según la Opción 2?

- ¿Qué porcentaje de conductores en línea completaron menos de 10 viajes, tuvieron una tasa de aceptación inferior al 90% y obtuvieron una calificación de 4,7 o superior?

**Escenario 2**

- ¿Cuánto dinero (después de gastos) gana el taxista al año sin asociarse con Uber?

- ¿Cuánto tendrían que aumentar las tarifas brutas del conductor por semana para pagar completamente el automóvil en el año 1 y mantener el mismo margen de ganancia anual que antes?

In [ ]:
# Acceso a google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sqlite3
import pandas as pd

# **ESCENARIO 1**

In [ ]:
'''Se define una función que conecta a la BD transporte y ejecuta las consultas SQL,
adicionalmente devuelve el resultado de la consulta como un DataFrame'''
def sql_consulta(instruccion):
  # Conexión a la base de datos
  conn = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/PROYECTOS/ANALISIS PLATAFORMA DE TRANSPORTE/transporte.db')
  cursor = conn.cursor()
  cursor.execute(instruccion)
  datos = cursor.fetchall()
  # Columnas
  col = [col[0] for col in cursor.description]
  # Crear un dataframe con los datos
  df = pd.DataFrame(datos, columns=col)
  conn.commit()
  conn.close()
  return df

In [ ]:
# Revisión de columnas de la base de datos
instruccion = '''SELECT *
FROM app
LIMIT 5'''
sql_consulta(instruccion)

,Name,TripsCompleted,AcceptRate,SupplyHours,Rating
0,Abdul,1,100%,3,4.8
1,Abraham,12,83%,5,4.7
2,Adelina,1,100%,2,4.7
3,Akilah,1,100%,2,4.9
4,Alec,21,76%,11,5.0


In [ ]:
instruccion = 'DROP VIEW IF EXISTS app_v2'
sql_consulta(instruccion)

In [ ]:
# Crear vista de la tabla app sin el símbolo %
instruccion= '''
CREATE VIEW app_v2 AS
SELECT Name,	TripsCompleted,	CAST(REPLACE(AcceptRate, \'%\', \'\') AS INT) AS AcceptRate,	SupplyHours,	Rating
FROM app
'''
sql_consulta(instruccion)

In [ ]:
instruccion = 'PRAGMA table_info(app_v2);'
sql_consulta(instruccion)

,cid,name,type,notnull,dflt_value,pk
0,0,Name,TEXT,0,None,0
1,1,TripsCompleted,INTEGER,0,None,0
2,2,AcceptRate,,0,None,0
3,3,SupplyHours,INTEGER,0,None,0
4,4,Rating,Float,0,None,0


In [ ]:
instruccion = 'SELECT * FROM app_v2 LIMIT 5;'
sql_consulta(instruccion)

,Name,TripsCompleted,AcceptRate,SupplyHours,Rating
0,Abdul,1,100,3,4.8
1,Abraham,12,83,5,4.7
2,Adelina,1,100,2,4.7
3,Akilah,1,100,2,4.9
4,Alec,21,76,11,5.0


In [ ]:
'''Cuenta la cantidad de conductores que cumplen las condiciones de la Opción 1
y se multiplica por el bono de $50'''

instruccion='''SELECT COUNT(Name) AS Bono
FROM app_v2
WHERE SupplyHours >= 8
AND TripsCompleted >= 10
AND AcceptRate >= 90
AND Rating >= 4.7'''

df=sql_consulta(instruccion)
result= df.iat[0,0]*50
print(f'El pago total del bono con la Opción 1 es de: ${result}')

El pago total del bono con la Opción 1 es de: $1050


In [ ]:
'''Se suma la cantidad de viajes para los conductores que cumplen la Opción2
y se multiplica por el bono de $4/viaje'''

instruccion='''SELECT SUM(TripsCompleted) AS Bono
FROM app_v2
WHERE TripsCompleted >= 12
AND Rating >= 4.7'''
df=sql_consulta(instruccion)
result= df.iat[0,0]*4
print(f'El pago total del bono con la Opción 2 es de: ${result}')

El pago total del bono con la Opción 2 es de: $2976


In [ ]:
# Consulta para buscar los conductores que cumplen con la Opción 1 y no con la Opción 2
instruccion=''' WITH Opcion1 AS (SELECT *
FROM app_v2
WHERE SupplyHours >= 8
AND TripsCompleted >= 10
AND AcceptRate >= 90
AND Rating >= 4.7),
Opcion2 AS (SELECT *
FROM app_v2
WHERE TripsCompleted >= 12
AND Rating >= 4.7)

SELECT Opcion1.Name, Opcion1.TripsCompleted,	Opcion1.AcceptRate,	Opcion1.SupplyHours,	Opcion1.Rating  FROM Opcion1
LEFT JOIN Opcion2
ON Opcion1.Name = Opcion2.Name
WHERE Opcion2.Name IS NULL'''

df=sql_consulta(instruccion)
result=df['Name'].count()
print(f'Hay {result} conductores que clasifican para el Bono de la Opción 1 y no clasifican para la Opción 2')
df


Hay 2 conductores que clasifican para el Bono de la Opción 1 y no clasifican para la Opción 2


,Name,TripsCompleted,AcceptRate,SupplyHours,Rating
0,Oren,11,91,9,4.8
1,Phyllis,10,90,8,4.8


In [ ]:
instruccion = '''
SELECT ROUND((CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM app_v2)) * 100, 2) AS porcentaje
FROM app_v2
WHERE TripsCompleted < 10
AND AcceptRate < 90
AND Rating >= 4.7
'''
df=sql_consulta(instruccion)

print(f'El porcentaje de conductores que completaron menos de 10 viajes, tuvieron una tasa de aceptación inferior al 90% y obtuvieron una calificación de 4,7 o superior es: {df.iat[0,0]}%')

El porcentaje de conductores que completaron menos de 10 viajes, tuvieron una tasa de aceptación inferior al 90% y obtuvieron una calificación de 4,7 o superior es: 10.92%


# **ESCENARIO 2**

**Beneficio con vehículo rentado**

In [ ]:
# Cálculo de ganancias y gastos por el total de semanas de trabajo en 1 año
semanas_año = 52
semanas_trabajo = semanas_año-3
ganancia_diaria = 200
ganancia_anual = ganancia_diaria*6*semanas_trabajo
gastos_seguro_mes = 400
gastos_alquiler_semana = 500
gastos_gas_semana = 200
gasto_anual = gastos_seguro_mes*12 + (gastos_alquiler_semana + gastos_gas_semana)*semanas_trabajo

In [ ]:
print(f'La ganancia anual es de ${ganancia_anual}')

La ganancia anual es de $58800


In [ ]:
print(f'El gasto anual es de ${gasto_anual}')

El gasto anual es de $39100


In [ ]:
# Cálculo del margen de beneficio anual sin afiliarse a la plataforma de transporte
beneficio = round((ganancia_anual-gasto_anual)/ganancia_anual * 100, 2)
print(f'El conductor gana anualmente después de gastos ${ganancia_anual-gasto_anual} que equivale a un margen de beneficio de {beneficio}%')

El conductor gana anualmente después de gastos $19700 que equivale a un margen de beneficio de 33.5%


**Beneficio plataforma de transporte**

In [ ]:
# Cálculo de gastos teniendo en cuenta la compra de un nuevo vehículo
vehiculo = 40000
gastos_seguro_mes = 400*0.8
gastos_gas_semana = 200*1.05
gasto_anual_vehiculo = vehiculo + (gastos_gas_semana*semanas_trabajo) + gastos_seguro_mes*12

In [ ]:
print(f'El gasto anual con la compra de un nuevo vehículo es de $ {gasto_anual_vehiculo}')

El gasto anual con la compra de un nuevo vehículo es de $ 54130.0


In [ ]:
beneficio_vehiculo = round((ganancia_anual-gasto_anual_vehiculo)/ganancia_anual * 100, 2)

In [ ]:
print(f'El beneficio con la compra de un nuevo vehículo es de {beneficio_vehiculo}%')

El beneficio con la compra de un nuevo vehículo es de 7.94%


In [ ]:
# Cálculo de la ganancia semanal con los nuevos gastos para obtener el mismo margen de beneficio del 33.5%
ganancia_semanal_vehiculo = round(gasto_anual_vehiculo/ (semanas_trabajo * 0.665), 2)
ganancia_semanal_vehiculo

1661.19

In [ ]:
print(f'La ganancia semanal teniendo en cuenta los gastos de afiliarse a la plataforma de transporte para obtener el mismo margen de beneficio es de ${ganancia_semanal_vehiculo}')

La ganancia semanal teniendo en cuenta los gastos de afiliarse a la plataforma de transporte para obtener el mismo margen de beneficio es de $1661.19
